# Sentiment Analysis with a LSTM Neural Network

In [2]:
import spacy
import numpy as np
import csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, SpatialDropout1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import  Sequential
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [3]:
nlp = spacy.load('en_core_web_sm')

## Data Wrangling
read the training data

In [4]:
path = "data/train.tsv"
with open(path, 'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    data = np.array(list(reader))

In [5]:
xtrain = data[1:,:][:,2]
ytrain = data[1:,:][:,3]
ytrain_categorical = to_categorical(ytrain) # one-hot encoding ytrain

In [6]:
# basic NLP: tokenization

In [7]:
MAX_NUM_WORDS = 16000
tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
tokenizer.fit_on_texts(xtrain)
sequences = tokenizer.texts_to_sequences(xtrain)

### padding

too short sentences:
    
`a really good --> a really good PAD PAD PAD PAD PAD .. maxlenght`

**TODO: try a smaller max length (10 tokens)**

In [18]:
MAX_SEQUENCE_LENGTH = len(max(xtrain, key = len))
xtrain_padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = "post")

In [34]:
word_index = tokenizer.word_index
EMBEDDING_DIM = 96 # vector length in spacy model

### Embedding

* look up all words from the vocabulary in spacy
* build a huge lookup dictionary {'word': vector}

In [35]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

In [23]:
for word, i in word_index.items():
    token = nlp(word)
    

In [33]:
embedding_vector.shape
embedding_matrix.shape
embedding_matrix[1].shape

(300,)

In [36]:
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    token = nlp(word)
    embedding_vector = token.vector  #getting vector form spacy
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### Model definition

In [37]:
K.clear_session()
model = Sequential([
    Embedding(num_words, EMBEDDING_DIM, weights=[embedding_matrix], \
              input_length=MAX_SEQUENCE_LENGTH, trainable=False), # using Spacy weights
    SpatialDropout1D(0.2), # regularization
    LSTM(16, # neurons for interpreting sentence converted to Word Vectors
         # 16 neurons working in parallel
         # TUNE THIS HYPERPARAMETER
         recurrent_activation='hard_sigmoid', 
         use_bias=True, 
         kernel_initializer='glorot_uniform', 
         recurrent_initializer='orthogonal', 
         bias_initializer='zeros',
         dropout=0.2, # regularization
         recurrent_dropout=0.2),
    # produce output probabilities (mutually exclusive, add up to 1)
    Dense(5, activation = "softmax"),
])


Loss functions:

* binary cross-entropy (logloss): binary classification
* categorical cross-entropy: mutually exclusive multiclass classification (with softmax)
* MSE - regression
* other - read it in research papers

In [38]:
# builds a computation graph
# here we could deploy on which CPU/GPU/server the model is running
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
# train for N iterations
iterations = 10 
model.fit(xtrain_padded, ytrain_categorical, epochs = iterations, validation_split=0.2, batch_size=1000, verbose = 1)

Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - ETA: 9:14 - loss: 1.6094 - accuracy: 0.04 - ETA: 7:33 - loss: 1.6086 - accuracy: 0.28 - ETA: 6:57 - loss: 1.6077 - accuracy: 0.36 - ETA: 6:29 - loss: 1.6070 - accuracy: 0.39 - ETA: 6:14 - loss: 1.6061 - accuracy: 0.42 - ETA: 6:07 - loss: 1.6053 - accuracy: 0.43 - ETA: 6:01 - loss: 1.6044 - accuracy: 0.44 - ETA: 5:57 - loss: 1.6035 - accuracy: 0.45 - ETA: 5:55 - loss: 1.6027 - accuracy: 0.46 - ETA: 5:53 - loss: 1.6017 - accuracy: 0.46 - ETA: 5:49 - loss: 1.6009 - accuracy: 0.46 - ETA: 5:45 - loss: 1.6000 - accuracy: 0.47 - ETA: 5:43 - loss: 1.5990 - accuracy: 0.47 - ETA: 5:40 - loss: 1.5979 - accuracy: 0.47 - ETA: 5:38 - loss: 1.5970 - accuracy: 0.47 - ETA: 5:41 - loss: 1.5960 - accuracy: 0.48 - ETA: 5:39 - loss: 1.5950 - accuracy: 0.48 - ETA: 5:36 - loss: 1.5940 - accuracy: 0.48 - ETA: 5:34 - loss: 1.5929 - accuracy: 0.48 - ETA: 5:31 - loss: 1.5917 - accuracy: 0.48 - ETA: 5:29

124848/124848 [==============================] - ETA: 8:18 - loss: 1.3215 - accuracy: 0.49 - ETA: 8:13 - loss: 1.2966 - accuracy: 0.50 - ETA: 8:56 - loss: 1.2927 - accuracy: 0.51 - ETA: 9:27 - loss: 1.2882 - accuracy: 0.51 - ETA: 9:42 - loss: 1.2899 - accuracy: 0.51 - ETA: 9:54 - loss: 1.2877 - accuracy: 0.51 - ETA: 9:50 - loss: 1.2831 - accuracy: 0.51 - ETA: 9:45 - loss: 1.2843 - accuracy: 0.51 - ETA: 9:50 - loss: 1.2835 - accuracy: 0.51 - ETA: 9:50 - loss: 1.2847 - accuracy: 0.51 - ETA: 9:40 - loss: 1.2857 - accuracy: 0.51 - ETA: 9:29 - loss: 1.2839 - accuracy: 0.51 - ETA: 9:26 - loss: 1.2825 - accuracy: 0.51 - ETA: 9:15 - loss: 1.2826 - accuracy: 0.51 - ETA: 9:03 - loss: 1.2835 - accuracy: 0.51 - ETA: 8:54 - loss: 1.2865 - accuracy: 0.51 - ETA: 8:44 - loss: 1.2865 - accuracy: 0.51 - ETA: 8:36 - loss: 1.2862 - accuracy: 0.51 - ETA: 8:27 - loss: 1.2855 - accuracy: 0.51 - ETA: 8:20 - loss: 1.2848 - accuracy: 0.51 - ETA: 8:13 - loss: 1.2838 - accuracy: 0.51 - ETA: 8:05 - loss: 1.2844 - 

124848/124848 [==============================] - ETA: 10:03 - loss: 1.2907 - accuracy: 0.510 - ETA: 9:53 - loss: 1.2751 - accuracy: 0.520 - ETA: 9:57 - loss: 1.2771 - accuracy: 0.51 - ETA: 10:01 - loss: 1.2822 - accuracy: 0.510 - ETA: 9:53 - loss: 1.2764 - accuracy: 0.514 - ETA: 9:51 - loss: 1.2758 - accuracy: 0.51 - ETA: 9:49 - loss: 1.2786 - accuracy: 0.51 - ETA: 9:45 - loss: 1.2781 - accuracy: 0.51 - ETA: 9:44 - loss: 1.2798 - accuracy: 0.51 - ETA: 9:50 - loss: 1.2803 - accuracy: 0.51 - ETA: 9:38 - loss: 1.2810 - accuracy: 0.51 - ETA: 9:26 - loss: 1.2790 - accuracy: 0.51 - ETA: 9:15 - loss: 1.2796 - accuracy: 0.51 - ETA: 9:07 - loss: 1.2820 - accuracy: 0.51 - ETA: 9:04 - loss: 1.2822 - accuracy: 0.51 - ETA: 8:58 - loss: 1.2809 - accuracy: 0.51 - ETA: 8:49 - loss: 1.2827 - accuracy: 0.51 - ETA: 8:43 - loss: 1.2814 - accuracy: 0.51 - ETA: 8:37 - loss: 1.2808 - accuracy: 0.51 - ETA: 8:34 - loss: 1.2794 - accuracy: 0.51 - ETA: 8:31 - loss: 1.2777 - accuracy: 0.51 - ETA: 8:27 - loss: 1.2

124848/124848 [==============================] - ETA: 11:38 - loss: 1.2660 - accuracy: 0.528 - ETA: 11:26 - loss: 1.2943 - accuracy: 0.508 - ETA: 11:13 - loss: 1.3071 - accuracy: 0.500 - ETA: 11:05 - loss: 1.3009 - accuracy: 0.504 - ETA: 10:55 - loss: 1.2965 - accuracy: 0.505 - ETA: 10:51 - loss: 1.2900 - accuracy: 0.511 - ETA: 10:45 - loss: 1.2852 - accuracy: 0.515 - ETA: 10:56 - loss: 1.2857 - accuracy: 0.515 - ETA: 10:59 - loss: 1.2790 - accuracy: 0.518 - ETA: 11:03 - loss: 1.2827 - accuracy: 0.515 - ETA: 11:03 - loss: 1.2833 - accuracy: 0.514 - ETA: 10:57 - loss: 1.2825 - accuracy: 0.514 - ETA: 10:53 - loss: 1.2811 - accuracy: 0.514 - ETA: 10:49 - loss: 1.2808 - accuracy: 0.514 - ETA: 10:52 - loss: 1.2826 - accuracy: 0.513 - ETA: 10:47 - loss: 1.2826 - accuracy: 0.512 - ETA: 10:40 - loss: 1.2833 - accuracy: 0.512 - ETA: 10:30 - loss: 1.2826 - accuracy: 0.512 - ETA: 10:23 - loss: 1.2824 - accuracy: 0.512 - ETA: 10:21 - loss: 1.2840 - accuracy: 0.511 - ETA: 10:21 - loss: 1.2845 - acc

124848/124848 [==============================] - ETA: 12:27 - loss: 1.2724 - accuracy: 0.518 - ETA: 12:23 - loss: 1.2857 - accuracy: 0.512 - ETA: 12:03 - loss: 1.2882 - accuracy: 0.512 - ETA: 11:55 - loss: 1.2898 - accuracy: 0.511 - ETA: 11:40 - loss: 1.2809 - accuracy: 0.515 - ETA: 11:31 - loss: 1.2848 - accuracy: 0.513 - ETA: 11:16 - loss: 1.2833 - accuracy: 0.512 - ETA: 11:21 - loss: 1.2829 - accuracy: 0.511 - ETA: 11:18 - loss: 1.2834 - accuracy: 0.510 - ETA: 11:07 - loss: 1.2875 - accuracy: 0.508 - ETA: 10:59 - loss: 1.2870 - accuracy: 0.509 - ETA: 10:47 - loss: 1.2906 - accuracy: 0.508 - ETA: 10:38 - loss: 1.2910 - accuracy: 0.507 - ETA: 10:33 - loss: 1.2887 - accuracy: 0.508 - ETA: 10:26 - loss: 1.2885 - accuracy: 0.508 - ETA: 10:19 - loss: 1.2878 - accuracy: 0.508 - ETA: 10:13 - loss: 1.2881 - accuracy: 0.508 - ETA: 10:07 - loss: 1.2872 - accuracy: 0.508 - ETA: 10:03 - loss: 1.2882 - accuracy: 0.507 - ETA: 9:57 - loss: 1.2878 - accuracy: 0.507 - ETA: 9:52 - loss: 1.2860 - accur

124848/124848 [==============================] - ETA: 11:45 - loss: 1.2887 - accuracy: 0.500 - ETA: 11:50 - loss: 1.2825 - accuracy: 0.506 - ETA: 11:51 - loss: 1.2745 - accuracy: 0.512 - ETA: 11:23 - loss: 1.2766 - accuracy: 0.511 - ETA: 11:23 - loss: 1.2826 - accuracy: 0.510 - ETA: 11:11 - loss: 1.2855 - accuracy: 0.510 - ETA: 11:01 - loss: 1.2828 - accuracy: 0.514 - ETA: 10:48 - loss: 1.2746 - accuracy: 0.519 - ETA: 10:45 - loss: 1.2799 - accuracy: 0.517 - ETA: 10:48 - loss: 1.2845 - accuracy: 0.514 - ETA: 10:48 - loss: 1.2860 - accuracy: 0.512 - ETA: 10:45 - loss: 1.2839 - accuracy: 0.514 - ETA: 10:44 - loss: 1.2857 - accuracy: 0.514 - ETA: 10:43 - loss: 1.2847 - accuracy: 0.514 - ETA: 10:37 - loss: 1.2844 - accuracy: 0.513 - ETA: 10:31 - loss: 1.2861 - accuracy: 0.512 - ETA: 10:26 - loss: 1.2846 - accuracy: 0.513 - ETA: 10:26 - loss: 1.2863 - accuracy: 0.512 - ETA: 10:23 - loss: 1.2857 - accuracy: 0.512 - ETA: 10:14 - loss: 1.2856 - accuracy: 0.512 - ETA: 10:06 - loss: 1.2842 - acc

124848/124848 [==============================] - ETA: 12:11 - loss: 1.3193 - accuracy: 0.492 - ETA: 11:51 - loss: 1.3137 - accuracy: 0.493 - ETA: 11:42 - loss: 1.3178 - accuracy: 0.490 - ETA: 11:32 - loss: 1.3054 - accuracy: 0.497 - ETA: 11:24 - loss: 1.3004 - accuracy: 0.500 - ETA: 11:15 - loss: 1.2961 - accuracy: 0.499 - ETA: 11:09 - loss: 1.2898 - accuracy: 0.503 - ETA: 11:05 - loss: 1.2886 - accuracy: 0.504 - ETA: 11:00 - loss: 1.2864 - accuracy: 0.506 - ETA: 10:54 - loss: 1.2876 - accuracy: 0.506 - ETA: 10:49 - loss: 1.2865 - accuracy: 0.506 - ETA: 10:42 - loss: 1.2847 - accuracy: 0.508 - ETA: 10:35 - loss: 1.2804 - accuracy: 0.510 - ETA: 10:29 - loss: 1.2799 - accuracy: 0.511 - ETA: 10:24 - loss: 1.2795 - accuracy: 0.511 - ETA: 10:19 - loss: 1.2800 - accuracy: 0.511 - ETA: 10:16 - loss: 1.2813 - accuracy: 0.511 - ETA: 10:10 - loss: 1.2813 - accuracy: 0.512 - ETA: 10:04 - loss: 1.2807 - accuracy: 0.512 - ETA: 9:58 - loss: 1.2815 - accuracy: 0.511 - ETA: 9:52 - loss: 1.2789 - accur

124848/124848 [==============================] - ETA: 11:44 - loss: 1.3143 - accuracy: 0.504 - ETA: 11:45 - loss: 1.2820 - accuracy: 0.514 - ETA: 11:45 - loss: 1.2875 - accuracy: 0.513 - ETA: 11:38 - loss: 1.2874 - accuracy: 0.513 - ETA: 11:31 - loss: 1.2779 - accuracy: 0.518 - ETA: 11:25 - loss: 1.2799 - accuracy: 0.519 - ETA: 11:23 - loss: 1.2834 - accuracy: 0.514 - ETA: 11:16 - loss: 1.2856 - accuracy: 0.512 - ETA: 11:11 - loss: 1.2847 - accuracy: 0.512 - ETA: 11:04 - loss: 1.2833 - accuracy: 0.513 - ETA: 10:58 - loss: 1.2796 - accuracy: 0.515 - ETA: 10:52 - loss: 1.2797 - accuracy: 0.514 - ETA: 10:46 - loss: 1.2764 - accuracy: 0.516 - ETA: 10:40 - loss: 1.2757 - accuracy: 0.516 - ETA: 10:35 - loss: 1.2737 - accuracy: 0.518 - ETA: 10:29 - loss: 1.2752 - accuracy: 0.517 - ETA: 10:23 - loss: 1.2776 - accuracy: 0.516 - ETA: 10:17 - loss: 1.2764 - accuracy: 0.516 - ETA: 10:11 - loss: 1.2767 - accuracy: 0.517 - ETA: 10:05 - loss: 1.2785 - accuracy: 0.516 - ETA: 9:59 - loss: 1.2789 - accu

124848/124848 [==============================] - ETA: 11:34 - loss: 1.2900 - accuracy: 0.494 - ETA: 12:45 - loss: 1.2839 - accuracy: 0.496 - ETA: 13:06 - loss: 1.2889 - accuracy: 0.503 - ETA: 12:25 - loss: 1.2869 - accuracy: 0.508 - ETA: 12:08 - loss: 1.2936 - accuracy: 0.503 - ETA: 11:45 - loss: 1.2947 - accuracy: 0.501 - ETA: 11:28 - loss: 1.2902 - accuracy: 0.504 - ETA: 11:19 - loss: 1.2883 - accuracy: 0.505 - ETA: 11:17 - loss: 1.2874 - accuracy: 0.507 - ETA: 11:10 - loss: 1.2892 - accuracy: 0.506 - ETA: 11:03 - loss: 1.2868 - accuracy: 0.508 - ETA: 10:57 - loss: 1.2833 - accuracy: 0.512 - ETA: 10:51 - loss: 1.2814 - accuracy: 0.513 - ETA: 10:41 - loss: 1.2828 - accuracy: 0.513 - ETA: 10:30 - loss: 1.2817 - accuracy: 0.513 - ETA: 10:21 - loss: 1.2817 - accuracy: 0.513 - ETA: 10:15 - loss: 1.2808 - accuracy: 0.514 - ETA: 10:10 - loss: 1.2834 - accuracy: 0.512 - ETA: 10:10 - loss: 1.2846 - accuracy: 0.511 - ETA: 10:03 - loss: 1.2832 - accuracy: 0.512 - ETA: 9:56 - loss: 1.2833 - accu

124848/124848 [==============================] - ETA: 12:05 - loss: 1.2871 - accuracy: 0.513 - ETA: 11:56 - loss: 1.2747 - accuracy: 0.517 - ETA: 12:09 - loss: 1.2776 - accuracy: 0.515 - ETA: 11:59 - loss: 1.2808 - accuracy: 0.510 - ETA: 11:54 - loss: 1.2773 - accuracy: 0.516 - ETA: 11:48 - loss: 1.2749 - accuracy: 0.516 - ETA: 11:53 - loss: 1.2771 - accuracy: 0.516 - ETA: 11:40 - loss: 1.2803 - accuracy: 0.515 - ETA: 11:23 - loss: 1.2779 - accuracy: 0.517 - ETA: 11:14 - loss: 1.2740 - accuracy: 0.521 - ETA: 11:14 - loss: 1.2730 - accuracy: 0.523 - ETA: 11:25 - loss: 1.2717 - accuracy: 0.522 - ETA: 11:37 - loss: 1.2708 - accuracy: 0.522 - ETA: 11:50 - loss: 1.2705 - accuracy: 0.523 - ETA: 12:03 - loss: 1.2699 - accuracy: 0.524 - ETA: 12:14 - loss: 1.2706 - accuracy: 0.522 - ETA: 12:22 - loss: 1.2699 - accuracy: 0.523 - ETA: 12:27 - loss: 1.2725 - accuracy: 0.522 - ETA: 12:29 - loss: 1.2749 - accuracy: 0.521 - ETA: 12:36 - loss: 1.2755 - accuracy: 0.521 - ETA: 12:36 - loss: 1.2766 - acc

In [242]:
model.save('kaggle01.h5')

In [ ]:
model.load('kaggle01.h5')

In [41]:
path = "data/test.tsv"
with open(path, 'rt') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    data = np.array(list(reader))
xtest = data[1:,:][:,2]

In [42]:
MAX_NUM_WORDS = 16000
# tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
# tokenizer.fit_on_texts(xtest)
#DONT RE-FIT THE TOKENIZER
sequences = tokenizer.texts_to_sequences(xtest)

#MAX_SEQUENCE_LENGTH = len(max(xtest, key = len))
xtest_padded = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = "post")

In [43]:
preds = model.predict(xtest_padded)

**TODO: check whether sentiment=2 is overrepresented in training data**

In [44]:
preds[9]

array([0.04572212, 0.17300108, 0.5120085 , 0.21014974, 0.05911857],
      dtype=float32)

In [47]:
d = [1,2,3]
d[1]=d
d

[1, [...], 3]